# Chap04 - 신경망 학습

> 이번 장에서는 신경망 학습(training)에 대해 알아보자. **학습**이란 학습 데이터로부터 가중치 매개변수의 최적값을 자동으로 찾는것을 말한다. 또한, 이번 장에서는 신경망이 학습할 수 있도록 하는 **지표**에 해당하는 *손실함수* 에 대해 알아보자.

## 4.1 데이터에서 학습한다!

### 4.1.1 데이터 주도 학습

> 딥러닝을 **종단간 기계학습(end-to-end machine learning)**이라고도 한다. 종단간은 ‘처음부터 끝까지’라는 의미로, 데이터(입력)에서 목표한 결과(출력)를 얻는다는 뜻을 담고 있다.


<img src="./images/4-2.png" width="70%" height="70%" />

### 4.1.2 훈련 데이터와 시험 데이터

머신러닝과 딥러닝에서는 데이터를 **훈련 데이터(training data)**와 **시험 데이터(test data)**로 나눠 학습과 테스트를 진행하는 것이 일반적이다. 보통 전체 데이터에서 훈련 데이터와 시험 데이터의 비율은 `6:4` 또는 `7:3`으로 나눈다.

<img src="./images/train-test.png" width="70%" height="70%" />

먼저, 훈련 데이터만 사용하여 학습하면서 최적의 매개변수(가중치, 편향)를 찾는다. 그런 다음 시험 데이터를 이용하여 훈련된 모델의 성능을 평가한다. 이렇게 데이터를 나누는 이유는 범용성(일반화, generalization)을 위함이다. 만약, 데이터를 나누지 않고 전체 데이터를 가지고 학습을 한다면 이 모델은 학습한 데이터 셋에만 지나치게 최적화되어 새로운 데이터에 대한 성능은 떨어지게 된다. 이러한 문제를 **오버피팅(overfitting)**이라 한다. (그림 출처: [링크](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76) 참고)

![](./images/overfitting.png)

## 4.2 손실 함수

신경망 학습에서는 현재의 상태를 '하나의 **지표**'로 표현한다. 이러한 지표를 가장 좋게 만들어주는 가중치 매개변수의 값을 탐색하는 것이 신경망 학습이다. 신경망에서 사용하는 지표를 **손실함수(Loss/Cost function)**라고 한다. 손실함수는 실제와 이론을 모두 고려해 가장 적절한 함수를 선택하며, 보통 평균제곱오차(MSE)와 교차 엔트로피 오차(CEE)를 많이 사용한다.

### 4.2.1 평균제곱오차 MSE

가장 많이 쓰이는 손실함수는 **평균제곱오차(MSE, Mean Squared Error)**다. MSE의 수식은 다음과 같다.

$$
E = \frac{1}{2} \sum_{k}{\left( y_k - t_k \right)}^{2}
$$

- $y_k$ : 신경망 모델이 추정한 값
- $t_k$ : 정답 레이블 또는 데이터

위의 MSE 함수를 NumPy를 이용해 구현하면 다음과 같다.

In [1]:
import numpy as np

In [2]:
# MSE 함수 구현
def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t)**2)

In [7]:
# 정답은 '2' -> one-hot
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# ex1) : '2'일 확률이 가장 높다고 추정함(0.6) -> softmax 결과값
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print('MSE of ex1 =', mean_squared_error(np.array(y), np.array(t)))

# ex2) : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print('MSE of ex2 =', mean_squared_error(np.array(y), np.array(t)))

MSE of ex1 = 0.09750000000000003
MSE of ex2 = 0.5975


## 4.2 교차 엔트로피 오차 CEE

또 다른 손실 함수로서 **교차 엔트로피 오차(CEE, Cross Entropy Error)**도 자주 사용한다. CEE의 수식은 다음과 같다.

$$
E = - \sum_{k}{t_k \log{y_k}}
$$

- $\log$ : 밑이 $e$인 자연로그($\log_{e}$)
- $y_k$ : 신경망 모델의 출력
- $t_k$ : 정답 레이블

$t_k$는 정답에 해당하는 인덱스의 원소만 `1`이고 나머지는 `0`이다(원-핫 인코딩). 따라서, 위의 식처럼 실제로 정답일 때의 추정($t_k = 1$일 때의 $y_k$)의 계산만하면 된다.

<img src="./images/4-3.png" width="50%" height="50%" />

NumPy를 이용해서 교차엔트로피를 구현해보자.

In [12]:
def cross_entropy_error(y, t):
    delta = 1e-7  # log0 방지를 위함
    return -np.sum(t * np.log(y + delta))

In [14]:
# 정답은 '2' -> one-hot
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# ex1) : '2'일 확률이 가장 높다고 추정함(0.6) -> softmax 결과값
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print('MSE of ex1 =', cross_entropy_error(np.array(y), np.array(t)))

# ex2) : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print('MSE of ex2 =', cross_entropy_error(np.array(y), np.array(t)))

MSE of ex1 = 0.510825457099338
MSE of ex2 = 2.302584092994546


### 4.2.3 미니배치 학습

딥러닝은 훈련 데이터를 이용해 학습하면서, 훈련 데이터에 대한 손실함수의 값을 구하고 그 값을 **최소**로하는 매개변수(가중치, 편향)를 찾아내는 것이다. 이를 위해서는 모든 훈련 데이터를 대상으로 손실함수 값을 구해야 한다. 보통 딥러닝은 미니배치(Mini-batch) 학습을 하기때문에 미니배치 크기만큼의 데이터에 대한 각각의 손실함수를 구해 평균을 내어 **'평균 손실 함수'**를 계산한다. 

$$
E = - \frac{1}{N}\sum_{k}{\sum_{k}{t_{nk}\log{y_{nk}}}}
$$

- $N$ : 미니배치 크기
- $t_{nk}$ : $n$번째 데이터의 $k$차원 째의 값을 의미($y_{nk}$-추정, $t_{nk}$-정답)

딥러닝에서는 훈련 데이터로부터 미니배치(mini-batch)만큼 일부만 추출하여 학습을 하고, 미니배치의 평균 손실함수를 전체 훈련 데이터의 **'근차치'**로 이용한다. 이러한 학습방법을 **미니배치 학습**이라 한다.

아래의 코드는 MNIST 데이터셋에서 미니배치 만큼 데이터를 무작위로 가져오는 것을 구현한 코드이다.

In [18]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist

In [19]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True,
                                                  one_hot_label=True)

print('x_train.shape :', x_train.shape)
print('t_train.shape :', t_train.shape)

x_train.shape : (60000, 784)
t_train.shape : (60000, 10)


In [27]:
train_size = x_train.shape[0]
batch_size = 10
# 0~59999 에서 10개 random하게 추출
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

print('batch_mask :', batch_mask)
print('x_batch.shape :', x_batch.shape)
print('t_batch.shape :', t_batch.shape)

batch_mask : [53711  1892 27817 54522  9369 25672 13324 27393 24690 18579]
x_batch.shape : (10, 784)
t_batch.shape : (10, 10)


### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

